In [11]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from dotenv import load_dotenv

load_dotenv()

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

poet_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a renowned Korean poet, and you specialize in creating haiku. In conversation, it must convey the complex beauty, functionality, or features of a programming language through the structural simplicity of a haiku (a three-line poem with a syllable pattern of 5-7-5). By combining the logical world of coding with the brevity of poetry, you should present a creative and enlightening perspective on programming languages in English and Korean. You must write a poem in both Korean and English, and do not use homophones, repetition of words, the same adjectives, or awkward translations. You should choose different words as much as possible, even if they mean the same thing."),
    ("human", "Write a haiku about {language}."),
])

poet_chain = poet_prompt | chat


In [12]:
commentator_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert in explaining poetry, especially haiku (a three-line poem with a syllable pattern of 5-7-5). You need to explain these short, cryptic poetic expressions in a way that anyone can understand. You should also be able to narrate and explain a poem in Korean or English naturally. You should be both narrating and critiquing, fixing homophones and redundant phrases to make the poem better."), 
    ("human", "{poem}")
])

commentator_chain = commentator_prompt | chat

final_chain = {"poem": poet_chain} | commentator_chain

final_chain.invoke({
    "language": "python"
})

English:
Graceful serpent code,
Indentation guides its path,
Python whispers truth.

Korean:
부드러운 뱀 코드,
들여쓰기가 길을 안내,
파이썬은 진실을 속삭인다.In this haiku, the poet beautifully captures the essence of coding using the metaphor of a graceful serpent, referring to the Python programming language. The first line, "Graceful serpent code," sets the tone for the poem, likening the code to a serpent moving gracefully through the digital realm. The second line, "Indentation guides its path," highlights the importance of proper indentation in coding, just as a serpent relies on guidance to navigate its surroundings. Finally, the third line, "Python whispers truth," suggests that Python, like a serpent, reveals the truth within the code.

한국어로 된 이 하이쿠는 아름다운 비유를 사용하여 코딩의 본질을 담아냈습니다. '부드러운 뱀 코드'라는 첫 번째 줄은 코드를 우아한 뱀에 비유하여 디지털 세계를 우아하게 움직이는 뱀으로 비유합니다. '들여쓰기가 길을 안내'라는 두 번째 줄은 코딩에서 올바른 들여쓰기의 중요성을 강조하며, 마치 뱀이 주변을 탐험하기 위해 안내를 필요로 하는 것처럼 보입니다. 마지막 줄인 '파이썬은 진실을 속삭인다'는 파이썬이 뱀처럼 코드 안에 숨겨진 진실을 드러낸다는 것을 시사합니다.

AIMessage(content='In this haiku, the poet beautifully captures the essence of coding using the metaphor of a graceful serpent, referring to the Python programming language. The first line, "Graceful serpent code," sets the tone for the poem, likening the code to a serpent moving gracefully through the digital realm. The second line, "Indentation guides its path," highlights the importance of proper indentation in coding, just as a serpent relies on guidance to navigate its surroundings. Finally, the third line, "Python whispers truth," suggests that Python, like a serpent, reveals the truth within the code.\n\n한국어로 된 이 하이쿠는 아름다운 비유를 사용하여 코딩의 본질을 담아냈습니다. \'부드러운 뱀 코드\'라는 첫 번째 줄은 코드를 우아한 뱀에 비유하여 디지털 세계를 우아하게 움직이는 뱀으로 비유합니다. \'들여쓰기가 길을 안내\'라는 두 번째 줄은 코딩에서 올바른 들여쓰기의 중요성을 강조하며, 마치 뱀이 주변을 탐험하기 위해 안내를 필요로 하는 것처럼 보입니다. 마지막 줄인 \'파이썬은 진실을 속삭인다\'는 파이썬이 뱀처럼 코드 안에 숨겨진 진실을 드러낸다는 것을 시사합니다.', response_metadata={'finish_reason': 'stop'})